# Interaktive plots af de estimerede valgsandsynligheder ud fra hver biltype

Tryk: "shift + enter" for at køre en kodeboks

In [14]:
# Importerer python-pakker og egen kode-filer
%matplotlib inline
import matplotlib.patches as mpatches
from ipywidgets import *
import matplotlib.pyplot as plt
import import_ipynb
from interactive_plot_code import interactive_el_cp, interactive_benzin_cp, interactive_diesel_cp, interactive_cp
import matplotlib as mpl

In [15]:
# Indstiller størrelsen på plottene
plt.rcParams['figure.figsize'] = (4, 3)
plt.rcParams['figure.dpi'] = 170
mpl.rc('xtick', labelsize = 6) 
mpl.rc('ytick', labelsize = 6)
plt.rcParams.update({'font.size': 6})

## Interaktivt plot for elbilerne

In [16]:
interact(interactive_el_cp, rate_1 = (0.0, 0.5, 0.01), rate_2 = (0.0, 1.7, 0.01), rate_3 = (0.0, 3.0, 0.01), reg_beløb_1 = (0.0, 77100.0, 100.0), reg_beløb_2 = (0.0, 240000.0, 100.0), bundfradrag = (0.0, 25700.0, 100.0), el_bundfradrag_1 = (147500.0, 170000.0, 100.0), el_fradrag_2 = (0.0, 1700.0, 100.0), indfasning = (0.0, 1.0, 0.1));


interactive(children=(FloatSlider(value=0.25, description='rate_1', max=0.5, step=0.01), FloatSlider(value=0.8…

## Interaktivt plot for benzinbilerne

In [17]:
interact(interactive_benzin_cp, rate_1 = (0.0, 0.5, 0.01), rate_2 = (0.0, 1.7, 0.01), rate_3 = (0.0, 3.0, 0.01), reg_beløb_1 = (0.0, 77100.0, 100.0), reg_beløb_2 = (0.0, 240000.0, 100.0), bundfradrag = (0.0, 25700.0, 100.0), el_bundfradrag_1 = (147500.0, 170000.0, 100.0), el_fradrag_2 = (0.0, 1700.0, 100.0), indfasning = (0.0, 1.0, 0.1));


interactive(children=(FloatSlider(value=0.25, description='rate_1', max=0.5, step=0.01), FloatSlider(value=0.8…

## Interaktivt plot for dieselbilerne

In [6]:
interact(interactive_diesel_cp, rate_1 = (0.0, 0.5, 0.01), rate_2 = (0.0, 1.7, 0.01), rate_3 = (0.0, 3.0, 0.01), reg_beløb_1 = (0.0, 77100.0, 100.0), reg_beløb_2 = (0.0, 240000.0, 100.0), bundfradrag = (0.0, 25700.0, 100.0), el_bundfradrag_1 = (147500.0, 170000.0, 100.0), el_fradrag_2 = (0.0, 1700.0, 100.0), indfasning = (0.0, 1.0, 0.1));


interactive(children=(FloatSlider(value=0.25, description='rate_1', max=0.5, step=0.01), FloatSlider(value=0.8…

## Interaktivt plot for alle biltyper

In [ ]:
interact(interactive_cp, rate_1 = (0.0, 0.5, 0.02), rate_2 = (0.0, 1.7, 0.02), rate_3 = (0.0, 3.0, 0.02), reg_beløb_1 = (0.0, 77100.0, 500.0), reg_beløb_2 = (0.0, 240000.0, 500.0), bundfradrag = (0.0, 25700.0, 250.0), el_bundfradrag_1 = (147500.0, 170000.0, 500.0), el_fradrag_2 = (0.0, 1700.0, 250.0), indfasning = (0.0, 1.0, 0.1));
